Inference Routine from a trained model

In [1]:
import json
import os
import torch
from argparse import ArgumentParser

from pytorch_lightning import seed_everything

from transformers import EarlyStoppingCallback

from src.datasetComposer import DatasetBuilder, composed_train_path, composed_test_path, compactComposer, test_path, train_path, test_path,setupTokenizer
from src.inference_utils import InferenceGenerator
from src.datasetHandlers import SmartCollator
from src.model_utils import get_basic_model
from src.trainerArgs import CustomTrainer, getTrainingArguments
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:

iterative_gen = True
composed_already = True

# Define the parameters used to set up the models
modeltype = 'iterative' if iterative_gen else 'normal'  # either baseline or 'earlyfusion'

# either t5-small,t5-base, t5-large, facebook/bart-base, or facebook/bart-large
modelbase = 'facebook/bart-base'

# we will use the above variables to set up the folder to save our model
pre_trained_model_name = modelbase.split(
    '/')[1] if 'bart' in modelbase else modelbase

# where the trained model will be saved
output_path = 'TrainModels/' + modeltype + '/'+pre_trained_model_name+'/'

#tests = json.load(open(test_path,encoding='utf-8'))

In [4]:
rand_seed = 453
seed_everything(rand_seed)
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

arguments = train_arguments = {'output_dir': output_path,
                               'warmup_ratio': 0.2,
                               #'disable_tqdm':False,
                               'per_device_train_batch_size': 8,
                               'num_train_epochs': 4,
                               'lr_scheduler_type': 'cosine',
                               'learning_rate': 5e-5,
                               'evaluation_strategy': 'steps',
                               'logging_steps': 500,
                               
                               'seed': rand_seed}

Global seed set to 453


In [5]:
from types import SimpleNamespace

In [6]:
class NarratorUtils:
    def __init__(self, modelbase, trained_model_path):
        self.modelbase = modelbase
        self.trained_model_path = trained_model_path
        # Setting up tokenizer
        self.tokenizer_ = setupTokenizer(modelbase)

        self.local_dict = SimpleNamespace(modelbase=modelbase,
                                          tokenizer_=self.tokenizer_)
        self.setup_performed = False

        print(' Dont forget to call initialise_Model() before running any inference')

    def initialise_Model(self):
        classification_explanator = get_basic_model(self.local_dict)()
        # Set up the model along with the tokenizers and other important stuff required to run the generation
        params_dict = json.load(open(self.trained_model_path+'/parameters.json'))
        #state_dict = json.load(open(args.model_base_dir+'/parameters.json'))
        best_check_point = params_dict['best_check_point']
        best_check_point_model = best_check_point + '/pytorch_model.bin'

        state_dict = torch.load(best_check_point_model)
        classification_explanator.load_state_dict(state_dict)

        classification_explanator.eval();
        self.setup_performed = True

        return classification_explanator


In [7]:

narrator_utils = NarratorUtils(modelbase,output_path)

 Dont forget to call initialise_Model() before running any inference


In [8]:
# initialise the model
classification_explanator = narrator_utils.initialise_Model()

In [9]:
import copy

import random
# Example of input

ml_task_name = 'Car Insurance Risk'

prediction_probabilities = {'Low': 0.76, 'High': 0.24}
# the features used to make the prediction
feature_names = ['Height', 'Mar_status', 'cur_loc', 'nb_friends', 'last_trip']

bcc = feature_names.copy()
random.shuffle(bcc)

# get the order and directions of influence from the explainable output from the XAI technique
# the methods expects the keys ['explanation_order','positives','negatives','ignore']
# 'positives' is the list of all the features with positive influence on the prediction decision and 'negatives' is the inverse.
# 'ignore' is the list of features identified as having very limited contribution to the prediction decision

attributions = {'explanation_order': ['Height', 'last_trip', 'cur_loc', 'nb_friends', 'Mar_status'],
                'positives': ['Height', 'last_trip', 'Mar_status'],
                'negatives': ['cur_loc', 'nb_friends'],
                'ignore': []
                
                 }


In [18]:
# if we want to generate the texts via the iterative generation then we have to define the style 
# We want our output text to first table about the prediction output
# step 1: talk about the feature order based on the attributions
# step 2: talk about the features with positive contributions to the decision
# step 3: ----- negative features
# step 4: ------- features with limited influence
# step 5: Make conclusion based on all the input information

# this will instruct the narrator to follow our desired output style
iterative_generation_steps = {'step 0': '',
                    'step 1': attributions['explanation_order'],
                    'step 2': attributions['positives'],
                    'step 3': attributions['negatives'],
                    'step 4': attributions['ignore'],
                    'step 5': '-'
                    }


full_text_generation_steps = {'step 0':''}


generation_instruction = iterative_generation_steps if iterative_gen else full_text_generation_steps

In [50]:
feature_names

['Height', 'Mar_status', 'cur_loc', 'nb_friends', 'last_trip']

In [94]:

from nltk.tokenize import sent_tokenize, word_tokenize

def inferenceIterativeGenerator(pack, 
                               pr_c=1,
                               ignore=False,
                               force_section=False,
                               include_full_set=False,
                               ):
    pack = pack_x = copy.deepcopy(pack)
    pack['narration'] = ' ,'.join([''.join(t) for t in  pack['steps']])
    outputs = pack['steps']

    max_init = 1
    results = []

    if len(outputs[max_init:]) > 0:
        sofar = ' '.join(outputs[:max_init])+' [N1S]'+' '
        pack_x = copy.deepcopy(pack)

        pack_x['next_sequence'] = sofar
        pack_x['prev_seq'] = '<prem>'
        results.append(pack_x)
    prev = copy.deepcopy(sofar)

    for idx, sent in enumerate(outputs[max_init:-1]):
        # print(idx+1)
        lotto = [0, 1, 1, 0, 0, 1]
        random.shuffle(lotto)
        pack_x = copy.deepcopy(pack)

        pack_x['next_sequence'] = sent+f' [N{idx+2}S]'
        pack_x['prev_seq'] = sofar
        results.append(pack_x)

        sofar += sent+f' [N{idx+2}S]'+' '
    pack_x = copy.deepcopy(pack)
    pack_x['prev_seq'] = sofar+' [EON]'
    pack_x['next_sequence'] = outputs[-1]+' [EON]'
    results.append(pack_x)

    if include_full_set:
        pack_f = copy.deepcopy(pack)
        pack_f['prev_seq'] = '<full_narration>' 
        pack_f['next_sequence'] = sofar+' [EON] '+ outputs[-1]+' [CON]' #outputs[-1]+' [EON]'
        results.append(pack_f)
    
    results.append(sofar+' [EON] ')
    return results

def processFeatureAttributions(attributions, narration, force_consistency=True, nb_base=None):

    #  nb_base: specifies the number of features to consider in the output text when full-text generation mode is used

    feature_division = attributions
    contradict, support, ignore = feature_division[
        'negatives'], feature_division['positives'], feature_division['ignore']

    output = {'features': [], 'order': [], 'direction': []}
    nat = set([c.strip() for c in word_tokenize(narration)])

    ordered_features = attributions['explanation_order']

    for pos,feat in enumerate(ordered_features):
        include = False

        # check if the user specified if this feat should be in the output text
        
        if feat in nat and force_consistency:
            include = True
        elif feat not in nat and force_consistency:
            include = False
        elif not force_consistency:
            include = True

        if nb_base is not None and nb_features < nb_base:
            include = True
        if include:
            output['features'].append(feat)
            output['order'].append(pos)
            if feat in contradict:
                output['direction'].append(-1)
            elif feat in support:
                output['direction'].append(1)
            else:
                output['direction'].append(-2)
        else:
            pass
    return output 


def  lineariseExplanation(data,
    attributions, randomise_preds=False, force_consistency=True, shrink=None):
    instance = data
    narration = cleanNarrations(copy.deepcopy(instance['next_sequence']))
    prev_seq = cleanNarrations(copy.deepcopy(instance['prev_seq']))
    full_narra = cleanNarrations(copy.deepcopy(instance['narration']))

    if len(narration) < 1:
        force_consistency = False
    feature_ranks =processFeatureAttributions(attributions, narration+prev_seq+full_narra, force_consistency=force_consistency)
    feature_ranks2 = processFeatureAttributions(attributions, narration, force_consistency=force_consistency)
    #processFeatureAttributions(attributions,nars[-1],force_consistency=iterative_gen)

    feature_desc, [pf, nf, neu_f], directions = linearisedFeaturesAttributions(
        feature_ranks, shrink=shrink)
    feature_desc2, [pf2, nf2, neu_f2], directions2 = linearisedFeaturesAttributions(
        feature_ranks2, shrink=shrink)

    preamble, place_holderss = processPredictionProbabilities2(instance['prediction_confidence_level'],
                                                               instance['predicted_class'], randomise=randomise_preds)

    preamble = preamble+' <|section-sep|> '+feature_desc

    class_labels = getClassLabels(7)
    class_dict = {f'C{i+1}': c for i, c in enumerate(class_labels)}
    class_dict['C1 or C2'] = '#CA or #CB'
    class_dict['C2 or C1'] = '#CB or #CA'
    class_dict.update({f'c{i+1}': c for i, c in enumerate(class_labels)})

    class_dict = place_holderss

    # [functools.reduce(lambda a, kv: a.replace(*kv), class_dict.items(),re.sub('\s+', ' ', ss.strip().replace('\n', ' '))) for ss in [preamble]][0]
    extended1 = preamble
    narr, prev = [functools.reduce(lambda a, kv: a.replace(*kv), class_dict.items(),
                                   re.sub('\s+', ' ', ss.strip().replace('\n', ' '))) for ss in [narration, prev_seq]]

    return {'rele_feat': [pf2, nf2, neu_f2], 'directions': directions, 'label_placeholders': place_holderss, 'preamble': extended1, 'narration': narr, 'prev_seq': prev,
            'positives': pf, 'negatives': nf, 'neutral': neu_f, 'pred_label': class_dict[instance['predicted_class']]}

In [95]:
list(generation_instruction.values())

['',
 ['Height', 'last_trip', 'cur_loc', 'nb_friends', 'Mar_status'],
 ['Height', 'last_trip', 'Mar_status'],
 ['cur_loc', 'nb_friends'],
 [],
 '-']

In [61]:
nars=inferenceIterativeGenerator({'steps':list([' '.join(t) for t in generation_instruction.values()]),
'prediction_confidence_level': ', '.join([ f'{k}:{round(v*100,2)}% ' for k,v in prediction_probabilities.items()]),
'predicted_class': 

},ignore= not iterative_gen)

In [96]:
nars[2]

{'steps': ['',
  'Height last_trip cur_loc nb_friends Mar_status',
  'Height last_trip Mar_status',
  'cur_loc nb_friends',
  '',
  '-'],
 'prediction_confidence_level': 'Low:76.0% , High:24.0% ',
 'predicted_class': 'Low',
 'narration': ',Height last_trip cur_loc nb_friends Mar_status,Height last_trip Mar_status,cur_loc nb_friends,,-',
 'next_sequence': 'Height last_trip Mar_status [N3S]',
 'prev_seq': ' [N1S] Height last_trip cur_loc nb_friends Mar_status [N2S] '}

In [97]:
pfa = processFeatureAttributions(attributions,nars[-1],force_consistency=iterative_gen)

In [98]:
pfa

{'features': ['Height', 'last_trip', 'cur_loc', 'nb_friends', 'Mar_status'],
 'order': [0, 1, 2, 3, 4],
 'direction': [1, 1, -1, -1, 1]}

In [80]:
class ExplanationRecord():
    def __init__(self,ml_task_name,feature_names,prediction_probabilities, attributions,iterative_mode=False) -> None:
        self.input_record = SimpleNamespace()
        classes = list(prediction_probabilities.keys())
        self.input_record.classes = classes
        self.input_record.feature_names= feature_names
        self.input_record.attributions = attributions
        self.input_record.ml_task_name = ml_task_name
        self.input_record.prediction_as_string =  ' , '.join([ f'{k}:{round(v*100,2)}% ' for k,v in prediction_probabilities.items()])

        predicted_label = max(prediction_probabilities, key=prediction_probabilities.get)
        self.input_record.predicted_class = predicted_label
        print(f"The ML model predicted the label : {predicted_label}")

        self.iterative_mode = iterative_mode


    def setup_generation_steps(self,generation_steps):
        if not self.iterative_mode:
            print(' The texts will be generated in the default full-text mode')
        self.input_record.generation_steps_as_string = list([' '.join(t) for t in generation_steps.values()])


In [81]:
exp_record = ExplanationRecord(ml_task_name,feature_names,prediction_probabilities, attributions,iterative_mode=True)
exp_record.setup_generation_steps(generation_instruction)

The ML model predicted the label : Low


In [88]:
#from src.NarrationDatautils_new import cleanNarrations


cleanNarrations = lambda x: x
nars=inferenceIterativeGenerator({'steps':exp_record.input_record.generation_steps_as_string,
'prediction_confidence_level': exp_record.input_record.prediction_as_string,
'predicted_class': exp_record.input_record.predicted_class

},ignore= not iterative_gen)

pfa = processFeatureAttributions(exp_record.input_record.attributions,nars[-1],force_consistency=iterative_gen)

instance = nars[3]
narration = cleanNarrations(copy.deepcopy(instance['next_sequence']))
prev_seq = cleanNarrations(copy.deepcopy(instance['prev_seq']))
full_narra = cleanNarrations(copy.deepcopy(instance['narration']))
feature_ranks =processFeatureAttributions(exp_record.input_record.attributions, narration+prev_seq+full_narra, force_consistency=True)
feature_ranks2 = processFeatureAttributions(exp_record.input_record.attributions, narration, force_consistency=True)

In [93]:
instance

{'steps': ['',
  'Height last_trip cur_loc nb_friends Mar_status',
  'Height last_trip Mar_status',
  'cur_loc nb_friends',
  '',
  '-'],
 'prediction_confidence_level': 'Low:76.0% , High:24.0% ',
 'predicted_class': 'Low',
 'narration': ',Height last_trip cur_loc nb_friends Mar_status,Height last_trip Mar_status,cur_loc nb_friends,,-',
 'next_sequence': 'cur_loc nb_friends [N4S]',
 'prev_seq': ' [N1S] Height last_trip cur_loc nb_friends Mar_status [N2S] Height last_trip Mar_status [N3S] '}

In [92]:
feature_ranks

{'features': ['Height', 'last_trip', 'cur_loc', 'nb_friends', 'Mar_status'],
 'order': [0, 1, 2, 3, 4],
 'direction': [1, 1, -1, -1, 1]}

In [91]:
feature_ranks2

{'features': ['cur_loc', 'nb_friends'], 'order': [2, 3], 'direction': [-1, -1]}